<a href="https://colab.research.google.com/github/JONNY-ME/Landslide-Prevention-and-Innovation-challenge/blob/main/Landslide_prediction_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a name = "Libraries"></a>
## 1. Import relevant libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install catboost --quiet

Mounted at /content/drive
     |████████████████████████████████| 76.8 MB 18 kB/s 


In [ ]:
# Import libraries
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

<a name = "Load"></a>
## 2. Load files

In [ ]:
# Read files to pandas dataframes
path = '/content/drive/MyDrive/landslide-prevention-and-innovation-challenge/'
Train = pd.read_csv(path+'Train.csv')
Test = pd.read_csv(path+'Test.csv')
sample_submission = pd.read_csv(path+'SampleSubmission.csv')

<a name = "Preview"></a>
## 3. Preview files

In [ ]:
data = pd.concat([Train.assign(train=1), Test.assign(train=0)])
print(data.shape)
data.head(5)

(16294, 228)


,Sample_ID,1_elevation,2_elevation,3_elevation,4_elevation,5_elevation,6_elevation,7_elevation,8_elevation,9_elevation,10_elevation,11_elevation,12_elevation,13_elevation,14_elevation,15_elevation,16_elevation,17_elevation,18_elevation,19_elevation,20_elevation,21_elevation,22_elevation,23_elevation,24_elevation,25_elevation,1_slope,2_slope,3_slope,4_slope,5_slope,6_slope,7_slope,8_slope,9_slope,10_slope,11_slope,12_slope,13_slope,14_slope,15_slope,16_slope,17_slope,18_slope,19_slope,20_slope,21_slope,22_slope,23_slope,24_slope,25_slope,1_aspect,2_aspect,3_aspect,4_aspect,5_aspect,6_aspect,7_aspect,8_aspect,9_aspect,10_aspect,11_aspect,12_aspect,13_aspect,14_aspect,15_aspect,16_aspect,17_aspect,18_aspect,19_aspect,20_aspect,21_aspect,22_aspect,23_aspect,24_aspect,25_aspect,1_placurv,2_placurv,3_placurv,4_placurv,5_placurv,6_placurv,7_placurv,8_placurv,9_placurv,10_placurv,11_placurv,12_placurv,13_placurv,14_placurv,15_placurv,16_placurv,17_placurv,18_placurv,19_placurv,20_placurv,21_placurv,22_placurv,23_placurv,24_placurv,25_placurv,1_procurv,2_procurv,3_procurv,4_procurv,5_procurv,6_procurv,7_procurv,8_procurv,9_procurv,10_procurv,11_procurv,12_procurv,13_procurv,14_procurv,15_procurv,16_procurv,17_procurv,18_procurv,19_procurv,20_procurv,21_procurv,22_procurv,23_procurv,24_procurv,25_procurv,1_lsfactor,2_lsfactor,3_lsfactor,4_lsfactor,5_lsfactor,6_lsfactor,7_lsfactor,8_lsfactor,9_lsfactor,10_lsfactor,11_lsfactor,12_lsfactor,13_lsfactor,14_lsfactor,15_lsfactor,16_lsfactor,17_lsfactor,18_lsfactor,19_lsfactor,20_lsfactor,21_lsfactor,22_lsfactor,23_lsfactor,24_lsfactor,25_lsfactor,1_twi,2_twi,3_twi,4_twi,5_twi,6_twi,7_twi,8_twi,9_twi,10_twi,11_twi,12_twi,13_twi,14_twi,15_twi,16_twi,17_twi,18_twi,19_twi,20_twi,21_twi,22_twi,23_twi,24_twi,25_twi,1_geology,2_geology,3_geology,4_geology,5_geology,6_geology,7_geology,8_geology,9_geology,10_geology,11_geology,12_geology,13_geology,14_geology,15_geology,16_geology,17_geology,18_geology,19_geology,20_geology,21_geology,22_geology,23_geology,24_geology,25_geology,1_sdoif,2_sdoif,3_sdoif,4_sdoif,5_sdoif,6_sdoif,7_sdoif,8_sdoif,9_sdoif,10_sdoif,11_sdoif,12_sdoif,13_sdoif,14_sdoif,15_sdoif,16_sdoif,17_sdoif,18_sdoif,19_sdoif,20_sdoif,21_sdoif,22_sdoif,23_sdoif,24_sdoif,25_sdoif,Label,train
0,1,130,129,127,126,123,126,125,124,122,119,122,121,119,117,115,119,117,115,114,112,116,114,113,111,110,35.26439,37.29208,33.85452,35.79576,40.31554,38.87666,39.50971,40.51059,45.83452,45.00000,36.05503,40.51059,44.56372,41.81031,38.87666,33.85452,38.87666,33.85452,33.85452,32.63194,32.63194,30.24626,30.24626,30.24626,30.24626,98.1301,113.1986,116.5650,123.6901,135.0000,97.12502,104.0362,110.5560,119.0546,126.8699,105.9454,110.5560,113.9625,116.5650,119.7449,116.5650,119.7449,116.5650,116.5650,128.6598,128.6598,120.9638,120.9638,120.9638,120.9638,0.038514,0.029463,0.031405,0.025771,0.010453,0.028321,0.027360,0.019831,0.009338,0.008806,0.021451,0.021244,0.017273,0.006708,-0.008671,0.023851,0.012586,-0.004249,0.003646,-0.006809,0.013328,0.005337,0.007039,-0.017753,-0.007777,0.013903,0.014674,0.018249,0.018368,0.003341,0.002023,0.005743,0.002238,-0.003820,-0.003302,-0.002141,0.003585,0.002025,-0.012269,-0.027184,0.000977,-0.012630,-0.031644,-0.025683,-0.037371,-0.018934,-0.021875,-0.023553,-0.048495,-0.039092,8.045186,8.333038,7.819405,8.032228,9.818933,9.295772,9.375107,9.499450,10.922910,11.001900,9.339861,9.883230,11.035840,10.298030,9.618946,8.703197,10.175750,8.651121,8.251134,8.074524,8.519887,7.989215,7.634287,7.804186,7.219216,3.173340,2.961406,3.315935,3.061250,3.246914,3.221661,3.153512,3.048637,2.915506,3.073973,3.765278,3.246664,3.154479,3.237765,3.392537,3.851345,3.673898,3.821337,3.584646,3.734637,4.003083,4.218082,3.990867,4.100921,3.715154,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,1.281767,1.281743,1.281708,1.281684,1.281649,1.281780,1.281757,1.281721,1.281698,1.281662,1.281789,1.281765,1.281730,1.281707,1.281671,1.281802,1.281779,1.281743,1.281720,1.281684,1.281811,1.281788,1.281752

In [ ]:
base_fts = ['elevation', 'lsfactor', 'placurv', 'procurv', 'sdoif', 'slope', 'twi', 'aspect']

df = pd.DataFrame()
for i in base_fts:
  cols = [x for x in data.columns if x.endswith(i)]
  df[f'{i}_mean'] = data[cols].mean(axis=1)
  df[f'{i}_max'] = data[cols].max(axis=1)
  df[f'{i}_min'] = data[cols].min(axis=1)
  df[f'{i}_std'] = data[cols].std(axis=1)
  df[f'{i}_skew'] = data[cols].skew(axis=1)
  df[f'{i}_sum'] = data[cols].sum(axis=1)
  df[f'{i}_sem'] = data[cols].sem(axis=1)
  df[f'{i}_kurtosis'] = data[cols].kurtosis(axis=1)

print(df.shape)
df.head()

(16294, 64)


,elevation_mean,elevation_max,elevation_min,elevation_std,elevation_skew,elevation_sum,elevation_sem,elevation_kurtosis,lsfactor_mean,lsfactor_max,lsfactor_min,lsfactor_std,lsfactor_skew,lsfactor_sum,lsfactor_sem,lsfactor_kurtosis,placurv_mean,placurv_max,placurv_min,placurv_std,placurv_skew,placurv_sum,placurv_sem,placurv_kurtosis,procurv_mean,procurv_max,procurv_min,procurv_std,procurv_skew,procurv_sum,procurv_sem,procurv_kurtosis,sdoif_mean,sdoif_max,sdoif_min,sdoif_std,sdoif_skew,sdoif_sum,sdoif_sem,sdoif_kurtosis,slope_mean,slope_max,slope_min,slope_std,slope_skew,slope_sum,slope_sem,slope_kurtosis,twi_mean,twi_max,twi_min,twi_std,twi_skew,twi_sum,twi_sem,twi_kurtosis,aspect_mean,aspect_max,aspect_min,aspect_std,aspect_skew,aspect_sum,aspect_sem,aspect_kurtosis
0,119.44,130,110,5.795688,0.164882,2986,1.159138,-1.046021,9.013694,11.035840,7.219216,1.124943,0.375556,225.342353,0.224989,-0.945658,0.012659,0.038514,-0.017753,0.014405,-0.289737,0.316465,0.002881,-0.567047,-0.008915,0.018368,-0.048495,0.018898,-0.426774,-0.222866,0.003780,-0.766957,1.281733,1.281811,1.281649,0.000046,-0.080964,32.043332,0.000009,-0.964816,36.830369,45.83452,30.24626,4.742594,0.275036,920.75922,0.948519,-0.844316,3.466513,4.218082,2.915506,0.390196,0.394862,86.662827,0.078039,-1.168838,116.864561,135.0000,97.12502,9.057176,-0.448383,2921.61402,1.811435,0.403625
1,156.20,162,150,4.092676,0.069766,3905,0.818535,-1.419892,8.013825,10.544930,5.661154,1.455476,-0.123580,200.345615,0.291095,-1.204373,0.010663,0.022271,-0.002641,0.007121,-0.184149,0.266576,0.001424,-1.102478,-0.003822,0.019445,-0.013503,0.007098,1.603080,-0.095545,0.001420,3.846539,1.359578,1.359683,1.359463,0.000056,-0.073524,33.989444,0.000011,-0.504428,28.524667,35.26439,21.80141,4.391785,-0.177093,713.11667,0.878357,-1.045678,4.660537,5.175152,4.109311,0.291070,0.123486,116.513413,0.058214,-0.593004,184.633436,198.4350,168.69010,8.046540,0.146889,4615.83590,1.609308,-0.450431
2,162.56,171,149,6.474823,-0.517681,4064,1.294965,-0.770406,10.958018,15.173530,6.171284,2.548048,-0.195168,273.950444,0.509610,-0.887080,0.028086,0.081775,-0.012906,0.021221,0.440298,0.702151,0.004244,0.608023,0.022452,0.080986,-0.015857,0.021865,0.918400,0.561299,0.004373,0.842830,1.365054,1.365161,1.364937,0.000058,-0.081683,34.126354,0.000012,-0.535188,38.336258,56.77032,21.80141,9.288387,0.288512,958.40645,1.857677,-0.562382,4.227799,5.951684,2.413693,0.981711,-0.191377,105.694982,0.196342,-0.789564,284.942436,310.6013,248.19860,17.485790,-0.284634,7123.56090,3.497158,-0.748418
3,76.40,80,72,2.565801,-0.180178,1910,0.513160,-1.180694,3.785720,6.316696,2.672297,0.916593,1.441685,94.642992,0.183319,1.478461,0.029064,0.041220,0.011739,0.008026,-0.362669,0.726599,0.001605,-0.770601,0.008895,0.021083,-0.012380,0.008533,-0.709054,0.222366,0.001707,0.527946,1.100818,1.101023,1.100630,0.000112,0.080622,27.520452,0.000022,-1.001312,19.271319,30.24626,15.79317,3.653959,1.563368,481.78297,0.730792,2.222137,3.737841,4.421484,3.015849,0.428677,-0.142723,93.446023,0.085735,-1.083842,173.076448,213.6901,135.00000,19.374545,-0.202386,4326.91120,3.874909,-0.291926
4,109.16,117,102,3.954744,0.103775,2729,0.790949,-0.647523,7.742521,10.569610,6.296785,1.006279,0.792694,193.563024,0.201256,1.214371,-0.004572,0.015373,-0.018709,0.008370,0.752786,-0.114302,0.001674,0.527901,-0.007679,0.009456,-0.022651,0.007231,0.258402,-0.191980,0.001446,0.508332,1.284221,1.284558,1.283876,0.000181,-0.045751,32.105513,0.000036,-0.712408,28.257938,32.63194,22.98977,2.706730,-0.362634,706.44844,0.541346,-0.826964,4.542060,5.617533,3.523277,0.632148,0.295712,113.551492,0.126430,-1.042539,125.512572,135.0000,116.56500,6.084374,0.291106,3137.81430,1.216875,-0.939511


In [ ]:
for i in ['geology']:
  cols = [x for x in data.columns if x.endswith(i)]
  df[f'{i}_count'] = data[cols].count(axis=1)
  df[f'{i}_quantile'] = data[cols].quantile(axis=1)
  df[f'{i}_mean'] = data[cols].mean(axis=1)

In [ ]:
main_cols = df.columns
main_cols = main_cols.append(data.columns.drop(['train', 'Label']))
data = pd.concat([data, df], axis=1)

data

,Sample_ID,1_elevation,2_elevation,3_elevation,4_elevation,5_elevation,6_elevation,7_elevation,8_elevation,9_elevation,10_elevation,11_elevation,12_elevation,13_elevation,14_elevation,15_elevation,16_elevation,17_elevation,18_elevation,19_elevation,20_elevation,21_elevation,22_elevation,23_elevation,24_elevation,25_elevation,1_slope,2_slope,3_slope,4_slope,5_slope,6_slope,7_slope,8_slope,9_slope,10_slope,11_slope,12_slope,13_slope,14_slope,15_slope,16_slope,17_slope,18_slope,19_slope,20_slope,21_slope,22_slope,23_slope,24_slope,25_slope,1_aspect,2_aspect,3_aspect,4_aspect,5_aspect,6_aspect,7_aspect,8_aspect,9_aspect,10_aspect,11_aspect,12_aspect,13_aspect,14_aspect,15_aspect,16_aspect,17_aspect,18_aspect,19_aspect,20_aspect,21_aspect,22_aspect,23_aspect,24_aspect,25_aspect,1_placurv,2_placurv,3_placurv,4_placurv,5_placurv,6_placurv,7_placurv,8_placurv,9_placurv,10_placurv,11_placurv,12_placurv,13_placurv,14_placurv,15_placurv,16_placurv,17_placurv,18_placurv,19_placurv,20_placurv,21_placurv,22_placurv,23_placurv,24_placurv,25_placurv,1_procurv,2_procurv,3_procurv,4_procurv,5_procurv,6_procurv,7_procurv,8_procurv,9_procurv,10_procurv,11_procurv,12_procurv,13_procurv,14_procurv,15_procurv,16_procurv,17_procurv,18_procurv,19_procurv,20_procurv,21_procurv,22_procurv,23_procurv,24_procurv,25_procurv,1_lsfactor,2_lsfactor,3_lsfactor,4_lsfactor,5_lsfactor,6_lsfactor,7_lsfactor,8_lsfactor,9_lsfactor,10_lsfactor,11_lsfactor,12_lsfactor,13_lsfactor,14_lsfactor,15_lsfactor,16_lsfactor,17_lsfactor,18_lsfactor,19_lsfactor,20_lsfactor,21_lsfactor,22_lsfactor,23_lsfactor,24_lsfactor,25_lsfactor,1_twi,2_twi,3_twi,4_twi,5_twi,6_twi,7_twi,8_twi,9_twi,10_twi,11_twi,12_twi,13_twi,14_twi,15_twi,16_twi,17_twi,18_twi,19_twi,20_twi,21_twi,22_twi,23_twi,24_twi,25_twi,1_geology,2_geology,3_geology,4_geology,5_geology,6_geology,7_geology,8_geology,9_geology,10_geology,11_geology,12_geology,13_geology,14_geology,15_geology,16_geology,17_geology,18_geology,19_geology,20_geology,21_geology,22_geology,23_geology,24_geology,25_geology,1_sdoif,2_sdoif,3_sdoif,4_sdoif,5_sdoif,6_sdoif,7_sdoif,8_sdoif,9_sdoif,10_sdoif,11_sdoif,12_sdoif,13_sdoif,14_sdoif,15_sdoif,16_sdoif,17_sdoif,18_sdoif,19_sdoif,20_sdoif,21_sdoif,22_sdoif,23_sdoif,24_sdoif,25_sdoif,Label,train,elevation_mean,elevation_max,elevation_min,elevation_std,elevation_skew,elevation_sum,elevation_sem,elevation_kurtosis,lsfactor_mean,lsfactor_max,lsfactor_min,lsfactor_std,lsfactor_skew,lsfactor_sum,lsfactor_sem,lsfactor_kurtosis,placurv_mean,placurv_max,placurv_min,placurv_std,placurv_skew,placurv_sum,placurv_sem,placurv_kurtosis,procurv_mean,procurv_max,procurv_min,procurv_std,procurv_skew,procurv_sum,procurv_sem,procurv_kurtosis,sdoif_mean,sdoif_max,sdoif_min,sdoif_std,sdoif_skew,sdoif_sum,sdoif_sem,sdoif_kurtosis,slope_mean,slope_max,slope_min,slope_std,slope_skew,slope_sum,slope_sem,slope_kurtosis,twi_mean,twi_max,twi_min,twi_std,twi_skew,twi_sum,twi_sem,twi_kurtosis,aspect_mean,aspect_max,aspect_min,aspect_std,aspect_skew,aspect_sum,aspect_sem,aspect_kurtosis,geology_count,geology_quantile,geology_mean
0,1,130,129,127,126,123,126,125,124,122,119,122,121,119,117,115,119,117,115,114,112,116,114,113,111,110,35.26439,37.29208,33.85452,35.795760,40.31554,38.87666,39.509710,40.510590,45.83452,45.00000,36.055030,40.51059,44.563720,41.81031,38.87666,33.854520,38.87666,33.85452,33.85452,32.631940,32.63194,30.24626,30.24626,30.24626,30.24626,98.13010,113.19860,116.56500,123.6901,135.00000,97.12502,104.036200,110.55600,119.05460,126.86990,105.9454,110.5560,113.96250,116.5650,119.7449,116.56500,119.74490,116.56500,116.565000,128.65980,128.65980,120.96380,120.96380,120.96380,120.96380,0.038514,0.029463,0.031405,0.025771,0.010453,0.028321,0.027360,0.019831,0.009338,0.008806,0.021451,0.021244,0.017273,0.006708,-0.008671,0.023851,0.012586,-0.004249,0.003646,-0.006809,0.013328,0.005337,0.007039,-0.017753,-0.007777,0.013903,0.014674,0.018249,0.018368,0.003341,0.002023,0.005743,0.002238,-0.003820,-0.003302,-0.002141,

In [ ]:
neighbours = [12, 18, 14, 8]

for ft in base_fts:
    for i in neighbours:
        data[f"{ft}_{i}_diff"] = data[f"13_{ft}"] - data[f"{i}_{ft}"]

In [ ]:
train_df = data[data.train == 1].drop(columns=['train'])
test_df = data[data.train == 0].drop(columns=['train', 'Label'])

train_df.shape, test_df.shape

((10864, 326), (5430, 325))

<a name = "Model"></a>
## 9. Model training

In [ ]:
import time 
from sklearn.metrics import f1_score

def squared_mean(results):
  mean_results = np.ones(results[0].shape)
  ln = len(results)
  for i in range(ln):
    mean_results *= results[i]

  return mean_results ** (1/ln)

class My_model:
    def __init__(self, models):
        self.models = models

    def fit_eval_pred(self, X, y, eval_set, val, test):
        results_eval = []
        results_test = []
        for i, model in enumerate(self.models):
            print(f"[Training]............................... Model_{i+1}")
            st = time.time()
            try:
                model.fit(X, y, eval_set=eval_set, verbose=False)
            except Exception as e:
                model.fit(X, y)
            print(f"[Prediction]............................. Model_{i+1}")
            p = model.predict_proba(val)
            print(f"Log loss = {f1_score(eval_set[0][1], np.argmax(p, axis=1))}", end=" "*6)
            results_eval.append(p)
            results_test.append(model.predict_proba(test))
            print(f"Time {time.time() - st :.2f}sec")

        # results_eval = np.mean(results_eval, axis=0)
        results_eval = squared_mean(results_eval)
        # results_test = np.mean(results_test, axis=0)
        results_test = squared_mean(results_test)

        return results_eval, results_test


In [ ]:
import time
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import imblearn.over_sampling as ovs


seed = 21

def get_models(seed=21):

    cat_params = {"iterations":10000, "learning_rate":0.09, "verbose":100, "random_state":seed,
            'use_best_model':True, 'early_stopping_rounds':100, }

    lgb_params = {'learning_rate':0.1, 'n_estimators':10000, 'random_state':seed,
            'early_stopping_round':100, 'verbose':-1, 'subsample':1., 'colsample_bytree':0.6}

    return [
            CatBoostClassifier(**cat_params),   
            LGBMClassifier(**lgb_params),
    ]


In [ ]:
X = train_df[test_df.columns]
y = train_df.Label
test = test_df[test_df.columns]


results = []
ns = 15
tot = 0

skf = StratifiedKFold(n_splits=ns, random_state=seed, shuffle=True)
for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(100*"-")
    print(f"Fold-{fold+1}")
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    
    # sampler = ovs.SMOTE(random_state=42)
    # X_train, y_train = sampler.fit_resample(X_train, y_train)

    main_model = My_model(get_models())
    pred, T_pred = main_model.fit_eval_pred(X_train, y_train, [(X_test, y_test)], X_test, test)

    score = f1_score(y_test, pd.Series([[0, 1][i >= .35] for i in pred[:, 1]]))

    print(f"f1 score = {score}")
    tot += score

    results.append(T_pred)

    print(100*"-")
    print("\n\n")

print(f"Average log loss = {tot / ns}")

----------------------------------------------------------------------------------------------------
Fold-1
[Training]............................... Model_1
[Prediction]............................. Model_1
Log loss = 0.668693009118541      Time 18.37sec
[Training]............................... Model_2
[Prediction]............................. Model_2
Log loss = 0.6948640483383686      Time 4.51sec
f1 score = 0.7096774193548387
----------------------------------------------------------------------------------------------------



----------------------------------------------------------------------------------------------------
Fold-2
[Training]............................... Model_1
[Prediction]............................. Model_1
Log loss = 0.7098765432098765      Time 24.44sec
[Training]............................... Model_2
[Prediction]............................. Model_2
Log loss = 0.7138461538461539      Time 4.96sec
f1 score = 0.7423822714681441
---------------------------

<a name = "Predictions"></a>
## 10. Test set predictions

In [ ]:
# 0.7280434714136
test_pred = pd.Series([[0, 1][i >= .29] for i in np.mean(results, axis=0)[:, 1]])
test_pred.value_counts()

0    3532
1    1898
dtype: int64

<a name = "Submission"></a>
## 11. Creating a submission file

In [ ]:
sub_file = pd.DataFrame({'Sample_ID': Test.Sample_ID, 'Label': test_pred})
sub_file.to_csv('/content/submission2.csv', index = False)
sub_file.head()

,Sample_ID,Label
0,10865,0
1,10866,0
2,10867,0
3,10868,1
4,10869,1
